In [629]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", 200)

from scipy.stats import shapiro

from haversine import haversine, Unit

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer, KNNImputer

from sklearn.cluster import DBSCAN

import matplotlib.pyplot as plt
import seaborn as sns

import datetime

print("Import complete")

Import complete


Utility methods to fill missing values for categorical columns

Reference: https://github.com/rvt123/Medium_Articles/blob/main/Data_Preprocessing_Reduce_Categories/ARTICLE_MEDIUM_DATA_PREPROCESSING_DECREASE_CATEGORY.ipynb

Might try KNN based imputation if this doesn't work

In [630]:
def find_index_(df,to_find,col):
    min_=1
    max_=len(df)
    to_find=to_find
    while min_<max_:
        mid=int((min_+max_)/2)
        if(df[1:mid][col].isna().sum()==to_find)and(df[col][(mid-1):mid].isna()).values:
            return mid
        elif(df[1:mid][col].isna().sum()>to_find)or(df[1:mid][col].isna().sum()==to_find):
            max_=mid-1
        else:
            min_=mid+1
        if max_==min_:
            if(df[1:min_][col].isna().sum()==to_find)and((df[col][(min_-1):min_].isna()).values):
                return min_

def replace_cat_list(df,col,cat_list):
    count_cat_dict_initial = {'Total_cat':0}
    for cat in cat_list:
        count_cat_dict_initial[cat] = df.loc[df[col]==cat,col].count()
        count_cat_dict_initial['Total_cat'] = count_cat_dict_initial.get('Total_cat') + count_cat_dict_initial[cat]
    count_cat_dict_initial['Total'] = len(df[col])
    count_cat_dict_final = {'Total_cat':0}
    for cat in cat_list[:-1]:
        count_cat_dict_final[cat] = int((count_cat_dict_initial.get(cat)/count_cat_dict_initial.get('Total_cat'))*count_cat_dict_initial.get('Total'))
        count_cat_dict_final['Total_cat'] = count_cat_dict_final.get('Total_cat') + count_cat_dict_final[cat]
    count_cat_dict_final[cat_list[-1]] = count_cat_dict_initial['Total'] - count_cat_dict_final['Total_cat']
    fill_dict = {}
    for cat in cat_list:
        fill_dict[cat] = count_cat_dict_final[cat] - count_cat_dict_initial[cat]
    for cat in cat_list[:-1]:
        fill_index = find_index_(df,fill_dict.get(cat),col)
        df.loc[0:fill_index,col] = df.loc[0:fill_index,col].fillna(cat)
    df.loc[:,col] = df.loc[:,col].fillna(cat_list[-1])
    return df

## Import Files

In [631]:
# test data
test = pd.read_csv(r"source/test_features.csv")

# train data
train = pd.read_csv(r"source/train_features.csv")

# target
labels = pd.read_csv(r"source/train_labels.csv")

# check whether rows are equal
print("train data => rows: %s, cols: %s" % (train.shape[0], train.shape[1]))
print("labels data => rows: %s, cols: %s" % (labels.shape[0], labels.shape[1]))
print("test data => rows: %s, cols: %s" % (test.shape[0], test.shape[1]))

assert(train.shape[1] == test.shape[1])

# heights column
fill_heights = pd.read_csv(r"source/heights.csv", index_col="id")

train data => rows: 59400, cols: 40
labels data => rows: 59400, cols: 2
test data => rows: 14850, cols: 40


## Pre-processing

In [632]:
# check for duplicates in data
train_dup_count = np.sum(train.duplicated())
label_dup_count = np.sum(labels.duplicated())
test_dup_count = np.sum(test.duplicated())

print("duplicates in train dataset: %s" % train_dup_count)
print("duplicates in label dataset: %s" % label_dup_count)
print("duplicates in test dataset: %s" % test_dup_count)

assert(train_dup_count == 0 and label_dup_count == 0 and test_dup_count == 0)

duplicates in train dataset: 0
duplicates in label dataset: 0
duplicates in test dataset: 0


In [633]:
# make id as index
train = train.set_index("id")
test = test.set_index("id")
labels = labels.set_index("id")

In [634]:
# differentiate train and test data
train["type"] = "train"
test["type"] = "test"

# create a data column by merging both train and label set
data = pd.concat([train, test], ignore_index=False)

# create train and label combo for visualization,
# then same transformation can be applied to data
visual = pd.merge(train, labels, on='id')

# encode status group
visual["status_group"] =  visual["status_group"].astype("category")
label_dict = dict(enumerate(visual["status_group"].cat.categories))

visual["status_group_codes"] =  visual["status_group"].cat.codes

assert (data.shape[0] == train.shape[0] + test.shape[0])
assert (data.shape[1] == train.shape[1] == test.shape[1])

# only train data is here. so drop type
visual.drop("type", axis=1, inplace=True)
assert (visual.shape[1] == train.shape[1] + labels.shape[1])

together = [visual, data]

Order columns by their count of unique values descending order

In [635]:
# displaying object column data
# both `train` and `test` have same columns

df = visual
col_details = []
for col in df.columns:
    if df[col].dtype in ["category", object]:
        col_details.append((col, df[col].dtype, df[col].nunique(), list(df[col].unique())))
col_details.sort(key=lambda x: 1 / x[-2])

temp = pd.DataFrame(col_details, columns=["Column", "Dtype", "N_Unique", "Unique_vals"])
temp["Column"]

0                  wpt_name
1                subvillage
2               scheme_name
3                 installer
4                      ward
5                    funder
6             date_recorded
7                       lga
8                    region
9           extraction_type
10    extraction_type_group
11        scheme_management
12               management
13                   source
14                    basin
15            water_quality
16    extraction_type_class
17                  payment
18             payment_type
19              source_type
20          waterpoint_type
21            quality_group
22    waterpoint_type_group
23         management_group
24                 quantity
25           quantity_group
26             source_class
27             status_group
28           public_meeting
29                   permit
30              recorded_by
Name: Column, dtype: object

This will be deleted in future.
Just for reference

12               management
13                   source
14                    basin
15            water_quality
16    extraction_type_class
17                  payment
18             payment_type
19              source_type
20          waterpoint_type
21            quality_group
22    waterpoint_type_group
23         management_group
24                 quantity
25           quantity_group
26             source_class
27           public_meeting
28                   permit
29              recorded_by

Start with `wpt_name` (Name of the waterpoint if there is one)

References:
1. https://github.com/drivendataorg/pump-it-up/blob/master/kamchatang/Water%20Pump%201%20-%20EDA%20and%20Data%20Cleaning.ipynb
2. https://stackoverflow.com/a/58434981/10582056

In [636]:
visual["wpt_name"].fillna("missing", inplace=True)
visual["wpt_name"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

none                       6.0%
Shuleni                    2.9%
Zahanati                   1.4%
Msikitini                  0.9%
Kanisani                   0.5%
                           ... 
Kwa Medadi                 0.0%
Kwa Kubembeni              0.0%
Shule Ya Msingi Milanzi    0.0%
Funua                      0.0%
Kwa Mzee Lugawa            0.0%
Name: wpt_name, Length: 37400, dtype: object

1. Majority of water points are without names.
2. Water points which have names are not significantly dominating

Hence this column can be dropped

In [637]:
for df in together:
    df.drop("wpt_name", axis=1, inplace=True)

Now `subvillage` (Geographic location)

References:
1. https://github.com/drivendataorg/pump-it-up/blob/master/kamchatang/Water%20Pump%201%20-%20EDA%20and%20Data%20Cleaning.ipynb
2. https://stackoverflow.com/a/58434981/10582056
3. https://github.com/sagol/pumpitup/blob/main/oof_model.ipynb

In [638]:
visual["subvillage"].fillna("missing", inplace=True)
visual["subvillage"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

Madukani        0.9%
Shuleni         0.9%
Majengo         0.8%
Kati            0.6%
missing         0.6%
                ... 
Kipompo         0.0%
Chanyamilima    0.0%
Ikalime         0.0%
Kemagaka        0.0%
Kikatanyemba    0.0%
Name: subvillage, Length: 19288, dtype: object

Same as above. Hence this column can be dropped

In [639]:
for df in together:
    df.drop("subvillage", axis=1, inplace=True)

Now `scheme_name`

In [640]:
visual["scheme_name"].fillna("missing", inplace=True)
visual["scheme_name"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

missing                    47.4%
K                           1.1%
None                        1.1%
Borehole                    0.9%
Chalinze wate               0.7%
                           ...  
Visiga water supplly        0.0%
Emanyata pipelines          0.0%
Magundi water supply        0.0%
Imalampaka water supply     0.0%
Mtawanya                    0.0%
Name: scheme_name, Length: 2697, dtype: object

1. About half of the names are missing
1. In rest of them, majority are without names.
2. No significantly dominating scheme name is there

Hence this column can be dropped

In [641]:
for df in together:
    df.drop("scheme_name", axis=1, inplace=True)

The `installer` values can be grouped together

Reference:

1. https://github.com/sagol/pumpitup/blob/main/oof_model.ipynb
2. https://github.com/drivendataorg/pump-it-up/blob/master/kamchatang/Water%20Pump%201%20-%20EDA%20and%20Data%20Cleaning.ipynb
3. https://stackoverflow.com/a/58434981/10582056

In [642]:
for df in together:
    df['installer'] = df['installer'].astype(str).str.lower()

    df['installer'].replace(
        to_replace=('fini water', 'fin water', 'finn water', 'finwater', 'finwate'),
        value='finw', inplace=True)

    df['installer'].replace(to_replace='jaica co', value='jaica', inplace=True)

    df['installer'].replace(
        to_replace=(
            'district water department', 'district water depar', 'district council',
            'district counci', 'village council orpha','kibaha town council',
            'village council', 'coun', 'village counil', 'council',
            'mbulu district council', 'counc', 'village council .oda',
            'sangea district coun', 'songea district coun', 'villege council',
            'district  council', 'quick win project /council', 'mbozi district council',
            'village  council', 'municipal council', 'tabora municipal council',
            'wb / district council'),
        value='council', inplace=True)

    df['installer'].replace(
        to_replace=(
            'rc church', 'rc churc', 'rcchurch/cefa', 'irc', 'rc', 'rc ch', 'hw/rc',
            'rc church/central gover', 'kkkt church', 'pentecost church', 'roman church',
            'rc/mission', 'rc church/cefa', 'lutheran church', 'tag church',
            'free pentecoste church of tanz', 'rc c', 'church', 'rc cathoric',
            'morovian church', 'cefa/rc church', 'rc mission', 'anglican church',
            'church of disciples', 'anglikana church', 'cetral government /rc',
            'pentecostal church', 'cg/rc', 'rc missionary', 'sda church', 'methodist church', 'trc',
            'rc msufi', 'haidomu lutheran church', 'baptist church', 'rc church brother',
            'st magreth church', 'anglica church', 'global resource co', 'rc mi',
            'baptist church of tanzania', 'fpct church', 'rc njoro', 'rc .church',
            'rc mis', 'batist church', 'churc', 'dwe/anglican church','missi', 'mission',
            'ndanda missions', 'rc/mission', 'cvs miss', 'missionaries', 'hydom luthelani',
            'luthe', 'haydom lutheran hospital', 'lutheran', 'missio', 'germany missionary',
            'grail mission kiseki bar', 'missionary', 'heri mission', 'german missionsry',
            'wamissionari wa kikatoriki', 'neemia mission', 'wamisionari wa kikatoriki'),
        value='church', inplace=True)

    df['installer'].replace(
        to_replace=(
            'central government', 'gove', 'central govt', 'gover', 'cipro/government',
            'governme', 'adra /government', 'isf/government', 'adra/government',
            'government /tcrs', 'village govt', 'government', 'government /community',
            'concern /government', 'goverm', 'village government', 'cental government',
            'govern', 'cebtral government', 'government /sda', 'tcrs /government',
            'tanzania government', 'centra govt', 'colonial government', 'misri government',
            'government and community', 'cetral government /rc', 'concern/government',
            'government of misri', 'lwi &central government', 'governmen', 'government/tcrs', 'government /world vision',
            'centra government'),
        value='tanzanian government', inplace=True)

    df['installer'].replace(
        to_replace=('world vission', 'world division', 'word divisio','world visiin'),
        value='world vision', inplace=True)

    df['installer'].replace(to_replace=('unicrf', 'unisef'), value='unicef', inplace=True)

    df['installer'].replace(
        to_replace=(
            'commu', 'olgilai village community', 'adra /community', 'adra/community',
            'rwe/ community', 'killflora /community', 'communit', 'taboma/community',
            'arab community', 'adra/ community', 'sekei village community', 'rwe/community',
            'arabs community', 'village community', 'government /community',
            'dads/village community', 'killflora/ community', 'mtuwasa and community',
            'rwe /community', 'ilwilo community', 'summit for water/community',
            'igolola community', 'ngiresi village community', 'rwe community',
            'african realief committe of ku', 'twesa /community', 'shelisheli commission',
            'twesa/ community', 'marumbo community', 'government and community',
            'community bank', 'kitiangare village community', 'oldadai village community',
            'twesa/community', 'tlc/community', 'maseka community', 'islamic community',
            'district community j', 'village water commission', 'village community members',
            'tcrs/village community', 'village water committee', 'comunity'),
        value='community', inplace=True)

    df['installer'].replace(
        to_replace=('danid', 'danda','danida co', 'danny', 'daniad', 'dannida', 'danids'),
        value='danida', inplace=True)

    df['installer'].replace(
        to_replace=('hesaws', 'huches', 'hesaw', 'hesawz', 'hesawq', 'hesewa'),
        value='hesawa', inplace=True)

    df['installer'].replace(
        to_replace=(
            'dwsp', 'kkkt _ konde and dwe', 'rwe/dwe', 'rwedwe', 'dwe/', 'dw', 'dwr',
            'dwe}', 'dwt', 'dwe /tassaf', 'dwe/ubalozi wa marekani', 'consultant and dwe',
            'dwe & lwi', 'ubalozi wa marekani /dwe', 'dwe&', 'dwe/tassaf', 'dw$',
            'dw e', 'tcrs/dwe', 'dw#', 'dweb', 'tcrs /dwe', 'water aid/dwe', 'dww'),
        value='dwe', inplace=True)

    df['installer'].replace(
        to_replace=(
            'africa muslim', 'muslimu society(shia)', 'africa muslim agenc',
            'african muslims age', 'muslimehefen international','islamic',
            'the isla', 'islamic agency tanzania',  'islam', 'nyabibuye islamic center'),
        value='muslims', inplace=True)

    df['installer'].replace(
        to_replace=(
            'british colonial government', 'british government', 'britain'),
        value='british', inplace=True)

    df['installer'].replace(
        to_replace=(
            'tcrs/tlc', 'tcrs /care', 'cipro/care/tcrs', 'tcrs kibondo', 'tcrs.tlc',
            'tcrs /twesa', 'tassaf /tcrs', 'tcrs/care', 'tcrs twesa', 'rwe/tcrs',
            'tcrs/twesa', 'tassaf/ tcrs', 'tcrs/ tassaf', 'tcrs/ twesa', 'tcrs a',
            'tassaf/tcrs'),
        value='tcrs', inplace=True)

    df['installer'].replace(
        to_replace=(
            'kkkt-dioces ya pare', 'kkkt leguruki', 'kkkt ndrumangeni', 'kkkt dme',
            'kkkt kilinga', 'kkkt canal', 'kkkt katiti juu', 'kkkt mareu'),
        value='kkkt', inplace=True)

    df['installer'].replace(to_replace=('norad/'), value='norad', inplace=True)

    df['installer'].replace( to_replace=('tasaf/dmdd', 'dmdd/solider'),value='dmdd', inplace=True)

    df['installer'].replace(to_replace=('cjejow construction', 'cjej0'), value='cjejow', inplace=True)

    df['installer'].replace(
        to_replace=('china henan constuction', 'china henan contractor', 'china co.', 'chinese'),
        value='china', inplace=True)

    df['installer'].replace(
        to_replace=(
            'local contract', 'local technician', 'local', 'local  technician',
            'locall technician', 'local te', 'local technitian', 'local technical tec',
            'local fundi', 'local technical', 'localtechnician', 'village local contractor',
            'local l technician'),
        value='local', inplace=True)

    df['installer'].replace(
        to_replace=(
            'oikos e .africa', 'oikos e.africa', 'africa amini alama',
            'africa islamic agency tanzania', 'africare', 'african development foundation',
            'oikos e. africa', 'oikos e.afrika', 'afroz ismail', 'africa', 'farm-africa',
            'oikos e africa', 'farm africa', 'africaone', 'tina/africare', 'africaone ltd',
            'african reflections foundation', 'africa m'),
        value='africa', inplace=True)

    df['installer'].replace(to_replace=('h', 'he', 'hsw'), value='hsw', inplace=True)

    df['installer'].replace(to_replace=('jaica', 'jica'), value='jaica', inplace=True)

    df['installer'].replace(to_replace=('lawatefuka water sup', 'losaa-kia water supp'), value='water sup', inplace=True)

    df['installer'].replace(to_replace=('water', 'water aid', 'water board'), value='water', inplace=True)

    df['installer'].replace(to_replace=('region water department', 'sengerema water department'), value='department', inplace=True)

    df['installer'].replace(to_replace=('oxfam', 'oxfarm'), value='oxfarm', inplace=True)

    df['installer'].replace(to_replace=('priva', 'private'), value='private', inplace=True)

    df['installer'].replace(to_replace=('twe', 'twesa'), value='twesa', inplace=True)

    df['installer'].replace(to_replace=('villa', 'villagers'), value='villagers', inplace=True)

    df['installer'].replace(to_replace=('wa', 'wachina', 'wananchi'), value='wa', inplace=True)

    df['installer'].replace(to_replace=('0', 'nan', '-', '_'), value='missing', inplace=True)

    df_installer_cnt = df.groupby('installer')['installer'].count()
    other_list = df_installer_cnt[df_installer_cnt<71].index.tolist()
    df['installer'].replace(to_replace=other_list, value='other', inplace=True)

In [643]:
visual["installer"].fillna("missing", inplace=True)
visual["installer"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

dwe                              30.5%
other                            14.2%
missing                           7.5%
tanzanian government              6.2%
community                         3.3%
danida                            2.8%
hesawa                            2.4%
council                           2.3%
rwe                               2.0%
church                            1.6%
kkkt                              1.6%
finw                              1.3%
tcrs                              1.2%
world vision                      1.2%
ces                               1.0%
amref                             0.7%
twesa                             0.7%
lga                               0.7%
tasaf                             0.7%
wedeco                            0.7%
dmdd                              0.7%
jaica                             0.6%
norad                             0.6%
unicef                            0.6%
oxfarm                            0.6%
hsw                      

From the result, it can be seen that following values dominate
1. dwe
2. other
3. tanzanian government
4. community
5. danida
6. hesawa
7. council
8. rwe
9. church
10. kkkt

There is also significant amount of `missing`: 7.5 %

So now reduce the total categories to these 11 values.

In [644]:
selected = ['dwe', 'tanzanian government', 'community', 'danida', 'hesawa', 'council', 'rwe', 'church', 'kkkt', 'missing']

def truncate(row):
    if row not in selected:
        return "other"

for df in together:
    df["installer"] = df.apply(lambda row: "other" if row["installer"] not in selected else row["installer"], axis=1)

# apply impute missing values
selected.remove("missing")

for df in together:
    df["installer"].replace(to_replace="missing", value=np.nan, inplace=True)
    df = replace_cat_list(df,'installer', selected)

In [645]:
piv_table = pd.pivot_table(visual,index=['installer','status_group'], values='status_group_codes', aggfunc='count')
piv_table

status_group_codes
installer            status_group                               
church               functional                              670
                     functional needs repair                  66
                     non functional                          227
community            functional                             1291
                     functional needs repair                  94
                     non functional                          591
council              functional                              543
                     functional needs repair                  93
                     non functional                          726
danida               functional                             1041
                     functional needs repair                  95
                     non functional                          545
dwe                  functional                            11906
                     functional needs repair                1987
                     non functional                         7961
hesawa               functional                              792
                     functional needs repair                  54
                     non functional                          557
kkkt                 functional                              807
                     functional needs repair                 149
                     non functional                          669
other                functional                            13638
                     functional needs repair                1318
                     non functional                         8673
rwe                  functional                              304
                     functional needs repair                 137
                     non functional                          765
tanzanian government functional                             1267
                     functional needs repair                 324
                     non functional                         2110

There is a clear distinction between values in `installer` and result

Next is `ward`(Geographic location)`

In [646]:
visual["ward"].fillna("missing", inplace=True)
visual["ward"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

# `ward` has no significant values. Hence this column can be dropped

for df in together:
    df.drop("ward", axis=1, inplace=True)

The `funder` values can be grouped together

Reference:
1. https://github.com/sagol/pumpitup/blob/main/oof_model.ipynb
2. https://github.com/drivendataorg/pump-it-up/blob/master/kamchatang/Water%20Pump%201%20-%20EDA%20and%20Data%20Cleaning.ipynb
3. https://stackoverflow.com/a/58434981/10582056

In [647]:
for df in together:
    df['funder'] = df['funder'].astype(str).str.lower()

    df['funder'].replace(
        to_replace=(
            'kkkt_makwale', 'kkkt-dioces ya pare', 'world vision/ kkkt', 'kkkt church',
            'kkkt leguruki', 'kkkt ndrumangeni', 'kkkt dme', 'kkkt canal', 'kkkt usa',
            'kkkt mareu'),
        value='kkkt', inplace=True)

    df['funder'].replace(
        to_replace=(
            'government of tanzania', 'norad /government', 'government/ community',
            'cipro/government', 'isf/government', 'finidagermantanzania govt',
            'government /tassaf', 'finida german tanzania govt', 'village government',
            'tcrs /government', 'village govt', 'government/ world bank',
            'danida /government', 'dhv/gove', 'concern /govern', 'vgovernment',
            'lwi & central government', 'government /sda', 'koica and tanzania government',
            'world bank/government', 'colonial government', 'misri government',
            'government and community', 'concern/governm', 'government of misri',
            'government/tassaf', 'government/school', 'government/tcrs', 'unhcr/government',
            'government /world vision', 'norad/government', 'ministry of water'),
        value='government', inplace=True)

    df['funder'].replace(
        to_replace=(
            'british colonial government', 'japan government', 'china government',
            'finland government', 'belgian government', 'italy government',
            'irish government', 'egypt government', 'iran gover', 'swedish', 'finland'),
        value='foreign government', inplace=True)

    df['funder'].replace(
        to_replace=(
            'rc church', 'anglican church', 'rc churc', 'rc ch', 'rcchurch/cefa',
            'irc', 'rc', 'churc', 'hw/rc', 'rc church/centr', 'pentecosta church',
            'roman church', 'rc/mission', "ju-sarang church' and bugango",
            'lutheran church', 'roman cathoric church', 'tag church ub', 'aic church',
            'free pentecoste church of tanz', 'tag church', 'fpct church', 'rc cathoric',
            'baptist church', 'morovian church', 'cefa/rcchurch', 'rc mission',
            'bukwang church saints', 'agt church', 'church of disciples', 'rc mofu',
            "gil cafe'church'", 'pentecostal church', 'bukwang church saint',
            'eung am methodist church', 'rc/dwe', 'cg/rc', 'eung-am methodist church',
            'rc missionary', 'sda church', 'methodist church', 'rc msufi',
            'haidomu lutheran church', 'nazareth church', 'st magreth church',
            'agape churc', 'rc missi', 'rc mi', 'rc njoro', 'world vision/rc church',
            'pag church', 'batist church', 'full gospel church', 'nazalet church',
            'dwe/anglican church', 'missi', 'mission', 'missionaries', 'cpps mission',
            'cvs miss', 'grail mission kiseki bar', 'shelisheli commission', 'missionary',
            'heri mission', 'german missionary', 'wamissionari wa kikatoriki',
            'rc missionary', 'germany missionary', 'missio', 'neemia mission', 'rc missi',
            'hydom luthelani', 'luthe', 'lutheran church',  'haydom lutheran hospital',
            'village council/ haydom luther', 'lutheran', 'haidomu lutheran church',
            'resolute golden pride project', 'resolute mininggolden pride',
            'germany cristians'),
        value='church', inplace=True)

    df['funder'].replace(
        to_replace=(
            'olgilai village community', 'commu', 'community', 'arab community',
            'sekei village community', 'arabs community', 'village community',
            'mtuwasa and community', 'ilwilo community', 'igolola community',
            'ngiresi village community', 'marumbo community', 'village communi',
            'comune di roma', 'comunity construction fund', 'community bank',
            "oak'zion' and bugango b' commu", 'kitiangare village community',
            'oldadai village community', 'tlc/community', 'maseka community',
            'islamic community',  'tcrs/village community', 'buluga subvillage community',
            'okutu village community', 'rural water supply and sanitat'),
        value='community', inplace=True)

    df['funder'].replace(
        to_replace=(
            'council', 'wb / district council', 'cdtfdistrict council',
            'sangea district council', 'mheza distric counc', 'kyela council',
            'kibaha town council', 'swidish', 'mbozi district council',
            'village council/ rose kawala',  'songea municipal counci',
            'quick win project /council', 'village council', 'villege council',
            'tabora municipal council', 'kilindi district co', 'kigoma municipal council',
            'district council', 'municipal council', 'district medical',
            'sengerema district council', 'town council', 'mkinga  distric cou',
            'songea district council', 'district rural project', 'mkinga distric coun',
            'dadis'),
        value='district', inplace=True)

    df['funder'].replace(
        to_replace=(
            'tcrs.tlc', 'tcrs /care', 'tcrst', 'cipro/care/tcrs', 'tcrs/care', 'tcrs kibondo'),
        value='tcrs', inplace=True)

    df['funder'].replace(
        to_replace=(
            'fini water', 'finw', 'fin water', 'finn water', 'finwater'),
        value='fini', inplace=True)

    df['funder'].replace(
        to_replace=(
            'islamic', 'the isla', 'islamic found', 'islamic agency tanzania',
            'islam', 'muislam', 'the islamic', 'nyabibuye islamic center', 'islamic society', 'african muslim agency',
            'muslims', 'answeer muslim grou', 'muslimu society(shia)',
            'unicef/african muslim agency', 'muslim world', 'muslimehefen international',
            'shear muslim', 'muslim society'),
        value='islam', inplace=True)

    df['funder'].replace(
        to_replace=('danida', 'ms-danish', 'unhcr/danida', 'tassaf/ danida'),
        value='danida', inplace=True)

    df['funder'].replace(
        to_replace=(
            'hesawa', 'hesawz', 'hesaw', 'hhesawa', 'hesawwa', 'hesawza', 'hesswa',
            'hesawa and concern world wide'),
        value='hesawa', inplace=True)

    df['funder'].replace(
        to_replace=('world vision/adra', 'game division', 'worldvision'),
        value='world vision', inplace=True)

    df['funder'].replace(
        to_replace=(
            'germany republi', 'a/co germany', 'aco/germany', 'bingo foundation germany',
            'africa project ev germany', 'tree ways german'),
        value='germany', inplace=True)

    df['funder'].replace( to_replace=('private', 'private individual'), value='private', inplace=True)

    df['funder'].replace(to_replace=('ces (gmbh)', 'ces(gmbh)'), value='ces', inplace=True)

    df['funder'].replace(to_replace=('concern', 'concern world wide'), value='concern', inplace=True)

    df['funder'].replace(to_replace=('jaica', 'jica'), value='concern', inplace=True)

    df['funder'].replace(to_replace=('jaica', 'jica'), value='concern', inplace=True)

    df['funder'].replace(to_replace=('lawatefuka water supply', 'magadini-makiwaru water', 'water', 'wateraid'), value='concern', inplace=True)

    df['funder'].replace(to_replace=('oxfam', 'oxfarm'), value='oxfarm', inplace=True)

    df['funder'].replace(to_replace=('0', 'nan', '-', '_'), value='missing', inplace=True)
    df_funder_cnt = df.groupby('funder')['funder'].count()
    other_list = df_funder_cnt[df_funder_cnt < 98].index.tolist()
    df['funder'].replace(to_replace=other_list, value='other', inplace=True)

In [648]:
visual["funder"].fillna("missing", inplace=True)
visual["funder"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

other                            17.8%
government                       16.7%
missing                           7.4%
danida                            5.3%
hesawa                            3.7%
concern                           3.5%
kkkt                              2.6%
district                          2.5%
church                            2.5%
rwssp                             2.3%
world bank                        2.3%
world vision                      2.1%
private                           1.9%
unicef                            1.8%
tasaf                             1.5%
dhv                               1.4%
dwsp                              1.4%
norad                             1.3%
fini                              1.3%
germany                           1.2%
tcrs                              1.0%
community                         0.9%
oxfarm                            0.9%
dwe                               0.8%
netherlands                       0.8%
hifab                    

From the result, it can be seen that following values dominate
1. other                            25.2%
2. government                       16.7%
3. danida                            5.3%
4. hesawa                            3.7%
5. concern                           3.5%
6. kkkt                              2.6%
7. district                          2.5%
8. church                            2.5%
9. rwssp                             2.3%
10. world bank                        2.3%
11. world vision                      2.1%

There is also significant amount of `missing`: 7.4 %

So now reduce the total categories to these 11 values.

In [649]:
selected = ['government', 'danida', 'hesawa', 'hesawa', 'concern',
            'kkkt', 'district', 'church', 'rwssp', 'world bank',
            'world vision', 'missing']

def truncate(row):
    if row not in selected:
        return "other"

for df in together:
    df["funder"] = df.apply(lambda row: "other" if row["funder"] not in selected else row["funder"], axis=1)

# apply impute missing values
selected.remove("missing")

for df in together:
    df["funder"].replace(to_replace="missing", value=np.nan, inplace=True)
    df = replace_cat_list(df,'funder', selected)

In [650]:
piv_table = pd.pivot_table(visual, index=['funder','status_group'], values='status_group_codes', aggfunc='count')
piv_table

status_group_codes
funder       status_group                               
church       functional                             1112
             functional needs repair                  73
             non functional                          274
concern      functional                             1366
             functional needs repair                 134
             non functional                          563
danida       functional                             1721
             functional needs repair                 159
             non functional                         1242
district     functional                              805
             functional needs repair                  63
             non functional                          620
government   functional                             6080
             functional needs repair                1106
             non functional                         6441
hesawa       functional                              943
             functional needs repair                 232
             non functional                         1042
kkkt         functional                              871
             functional needs repair                  82
             non functional                          593
other        functional                            16892
             functional needs repair                2046
             non functional                        10247
rwssp        functional                              805
             functional needs repair                 109
             non functional                          460
world bank   functional                              545
             functional needs repair                  97
             non functional                          707
world vision functional                             1119
             functional needs repair                 216
             non functional                          635

Next is `date_recorded` (The date the row was entered)

The column can be converted to date object. Then `year`
can be extracted from it and the `age` of each pump can be calculated

In [651]:
visual["date_recorded"].describe()

# date is in format: 2011-03-15 year-month-date
# extract year from this column

count          59400
unique           356
top       2011-03-15
freq             572
Name: date_recorded, dtype: object

In [652]:
def calculate_age(row):
    date_recorded = datetime.datetime.strptime(str(row['date_recorded']), '%Y-%m-%d')
    year_recorded = int(date_recorded.strftime('%Y'))
    construction_year = row['construction_year']
    if year_recorded > construction_year > 0:
        return np.abs(year_recorded - construction_year)
    else: return np.nan


for df in together:
    df["age"] = df.apply(calculate_age, axis=1)

# note that `age` has nan values which have to be filled later

# now drop `date_recorded`
for df in together:
    df.drop("date_recorded", axis=1, inplace=True)

`construction_year` can be binned

Reference: https://github.com/drivendataorg/pump-it-up/blob/master/kamchatang/Water%20Pump%201%20-%20EDA%20and%20Data%20Cleaning.ipynb

In [653]:
def construction_wrangler(row):
    if 1960 <= row['construction_year'] < 1970:
        return '60s'
    elif 1970 <= row['construction_year'] < 1980:
        return '70s'
    elif 1980 <= row['construction_year'] < 1990:
        return '80s'
    elif 1990 <= row['construction_year'] < 2000:
        return '90s'
    elif 2000 <= row['construction_year'] < 2010:
        return '00s'
    elif row['construction_year'] >= 2010:
        return '10s'
    else:
        return "missing"

selected = ["60s", "70s", "80s", "90s", "00s", "10s"]

# apply impute missing values
for df in together:
    df['construction_year'] = df.apply(lambda row: construction_wrangler(row), axis=1)
    # df["construction_year"].replace(to_replace="missing", value=np.nan, inplace=True)
    # df = replace_cat_list(df,'construction_year', selected)

Next is `region` and `lga`.

There is not one dominating value in here. So drop these columns.

In [654]:
visual["region"].fillna("missing", inplace=True)
visual["region"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

Iringa           8.9%
Shinyanga        8.4%
Mbeya            7.8%
Kilimanjaro      7.4%
Morogoro         6.7%
Arusha           5.6%
Kagera           5.6%
Mwanza           5.2%
Kigoma           4.7%
Ruvuma           4.4%
Pwani            4.4%
Tanga            4.3%
Dodoma           3.7%
Singida          3.5%
Mara             3.3%
Tabora           3.3%
Rukwa            3.0%
Mtwara           2.9%
Manyara          2.7%
Lindi            2.6%
Dar es Salaam    1.4%
Name: region, dtype: object

In [655]:
visual["lga"].fillna("missing", inplace=True)
visual["lga"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

for df in together:
    df.drop(["region", "lga"], axis=1, inplace=True)

Next `scheme_management`

In [656]:
visual["scheme_management"].fillna("missing", inplace=True)
visual["scheme_management"].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

VWC                 61.9%
WUG                  8.8%
missing              6.5%
Water authority      5.3%
WUA                  4.9%
Water Board          4.6%
Parastatal           2.8%
Private operator     1.8%
Company              1.8%
Other                1.3%
SWC                  0.2%
Trust                0.1%
None                 0.0%
Name: scheme_management, dtype: object

VWC                 61.9%
WUG                  8.8%
missing              6.5%
Water authority      5.3%
WUA                  4.9%
Water Board          4.6%

Truncate others

In [657]:
selected = ['VWC', 'WUG', 'Water authority', 'WUA', 'Water Board', 'missing']

def truncate(row):
    if row not in selected:
        return "other"

for df in together:
    df["scheme_management"] = df.apply(lambda row: "other" if row["scheme_management"] not in selected else row["scheme_management"], axis=1)

Sanitizing `null` values

For boolean columns: -> fill with median value

In [658]:
# replace boolean cols with median values
data["public_meeting"].fillna("missing", inplace=True)
data["permit"].fillna("missing", inplace=True)

For numeric columns

Reference: https://github.com/villeheilala/pumpitup/blob/master/pumpitup_preprocess.ipynb

In [659]:
data["amount_tsh"] = data["amount_tsh"].apply(lambda x: np.log(round(x)) if round(x) > 0 else 0)
data["population"] = data["population"].apply(lambda x: np.log(x) if x > 0 else 0)

age_mean = data["age"].mean()
data["age"].fillna(value=age_mean, inplace=True)

For `gps_height`, fill with reference data

Reference: https://github.com/villeheilala/pumpitup/blob/master/heights.csv

In [660]:
data["gps_height"].replace(to_replace=0, value=np.nan, inplace=True)
data["gps_height"] = data["gps_height"].fillna(fill_heights["gps_height"])
data["gps_height"].fillna(value=data["gps_height"].mean(), inplace=True)

assert (data["gps_height"].isnull().sum() == 0)

Fill missing `latitude` and `longitude` data

Because geocode from google requires payment, this alternative is suitable

Reference: https://github.com/sagol/pumpitup/blob/main/oof_model.ipynb

In [661]:
data_geo = data.groupby(['region_code'])[['latitude', 'longitude']].median()

def geo_update(row, df_geo):
    row['longitude'] = df_geo.loc[row['region_code']]['longitude']
    row['latitude'] = df_geo.loc[row['region_code']]['latitude']
    return row

data.loc[data['longitude']== 0, ['longitude', 'latitude']] \
    = data[data['longitude']==0].apply(
        geo_update,
        df_geo=data_geo,
        axis=1)[['longitude', 'latitude']]

Working with `Latitude` and `Longitude`

Reference: https://stackoverflow.com/a/31398615/10582056

1. Find the haversine distance

In [662]:
mean_lat = data["latitude"].mean()
mean_long = data["longitude"].mean()

data["haversine_distance"] = data.apply(lambda row: haversine((row["latitude"], row["longitude"]), (mean_lat, mean_long), unit=Unit.KILOMETERS), axis=1)

2. Convert `latitude`, `longitude` to `x_coordinate`, `y_coordinate` and `z_coordinate`

Reference: https://heartbeat.fritz.ai/working-with-geospatial-data-in-machine-learning-ad4097c7228d

In [663]:
data['x_coordinate'] = np.cos(data['latitude']) * np.cos(data['longitude'])
data['y_coordinate'] = np.cos(data['latitude']) * np.sin(data['longitude'])
data['z_coordinate'] = np.sin(data['latitude'])

3. Do clustering with DBSCAN

Reference: https://github.com/villeheilala/pumpitup/blob/master/pumpitup_preprocess.ipynb

In [664]:
db = DBSCAN(eps=0.2, min_samples=200)
data["location_cluster"] = db.fit_predict(data[["latitude", "longitude"]])

print("Clustering complete")

Clustering complete


Drop unnecessary / similar columns

Reference:
1. https://github.com/villeheilala/pumpitup/blob/master/pumpitup_preprocess.ipynb
2. https://github.com/sagol/pumpitup/blob/main/oof_model.ipynb

In [665]:
similar_cols = ['recorded_by', "latitude", "longitude",
                'extraction_type', 'extraction_type_group',
                 "management", "management_group",
                 "payment",
                 "quality_group",
                 "source",
                 "num_private", "district_code", "region_code"]

data.drop(similar_cols, axis=1, inplace=True)

Data types in `data` are mixed. So transform

1. Some columns are in int format, but they are just categories -> convert them to string

Reference: https://github.com/villeheilala/pumpitup/blob/master/pumpitup_preprocess.ipynb

2. computable `numeric` types to `float64`

3. string columns 'category' for quick transformations

In [666]:
categorical_columns = [col for col in data.columns if data[col].dtype in [object, "category"]] + ["location_cluster"]
for col in categorical_columns:
    data[col] = data[col].map(lambda x: str(x).lower()).astype("category")

# numeric columns
numeric_columns = [col for col in data.columns if col not in categorical_columns]

# convert to float64
for col in numeric_columns:
    data[col] = data[col].astype("float64")

In [667]:
df = data
col_details = []
for col in df.columns:
    col_details.append((col, df[col].dtype, df[col].nunique(), list(df[col].unique()), df[col].isnull().sum()))
col_details.sort(key=lambda x: 1 / x[-3])

temp = pd.DataFrame(col_details, columns=["Column", "Dtype", "N_Unique", "Unique_vals", "Null values"])
temp

,Column,Dtype,N_Unique,Unique_vals,Null values
0,haversine_distance,float64,71877,"[451.9875545518294, 407.71706910655627, 342.79...",0
1,x_coordinate,float64,71877,"[0.8433323014130844, 0.5398050788475607, -0.75...",0
2,y_coordinate,float64,71877,"[0.3373986455318504, 0.07675693112137497, 0.18...",0
3,z_coordinate,float64,71871,"[0.4182735748126857, -0.8382832757339295, 0.62...",0
4,gps_height,float64,2481,"[1390.0, 1399.0, 686.0, 263.0, 1671.0, 6.0, 11...",0
5,population,float64,1127,"[4.6913478822291435, 5.634789603169249, 5.5214...",0
6,amount_tsh,float64,98,"[8.699514748210191, 0.0, 3.2188758248682006, 2...",0
7,age,float64,54,"[12.0, 3.0, 4.0, 27.0, 15.535065208245689, 2.0...",0
8,location_cluster,category,31,"[5, 6, 4, -1, 0, 1, 26, 2, 3, 17, 7, 25, 8, 9,...",0
9,funder,category,11,"[other, district, rwssp, concern, danida, worl...",0


Check if numeric columns follows Gaussian pattern
if true: use `StandardScalar` else use `MinMaxScalar`

Reference: https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/

In [668]:
def is_gaussian(feature, alpha):
    stat, p = shapiro(data[feature])
    print('%s: statistics=%.3f, p=%.3f, %s' % (feature, stat, p, 'yes' if p > alpha else 'no'))

for column in numeric_columns:
    is_gaussian(column, 0.05)

amount_tsh: statistics=0.642, p=0.000, no
gps_height: statistics=0.943, p=0.000, no
population: statistics=0.783, p=0.000, no
age: statistics=0.897, p=0.000, no
haversine_distance: statistics=0.988, p=0.000, no
x_coordinate: statistics=0.971, p=0.000, no
y_coordinate: statistics=0.967, p=0.000, no
z_coordinate: statistics=0.922, p=0.000, no


E:\OpenSoftware\miniconda3\envs\pump_it_up\lib\site-packages\scipy\stats\morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


Doing standardisation with `MinMax Scalar`

In [ ]:
standard_cols = [
    'amount_tsh',
    'gps_height',
    'population',
    'haversine_distance',
    'x_coordinate',
    'y_coordinate',
    'z_coordinate',
    'age'
]

scalar = MinMaxScaler()
data[standard_cols] = scalar.fit_transform(data[standard_cols])

Make all strings to lowercase

In [ ]:
for col in categorical_columns:
    data[col] = data[col].apply(lambda x: x.lower(), convert_dtype=False)